In [1]:
import pandas as pd
import numpy as np
import datetime
import quandl
import config


# settings to observe x amount of rows in dataframe
pd.set_option('max_rows', 500)
pd.set_option('max_columns', 1000)

# API Key
key = quandl.ApiConfig.api_key

### OPENS FILES

In [2]:
# North America Publicly Traded Companies
companies_data = pd.read_csv('/Users/Alfredo/Desktop/clustering-stocks/n_a_companylist.csv')
companies_data.head(1)

,Symbol,Name,LastSale,MarketCap,ADR TSO,IPOyear,Sector,Industry,Summary Quote,Unnamed: 9
0,DDD,3D Systems Corporation,9.15,1.041840e+09,NaN,NaN,Technology,Computer Software: Prepackaged Software,http://www.nasdaq.com/symbol/ddd,NaN


In [3]:
companies_data.shape

(2457, 10)

In [4]:
# Quandl Available Companies
tickers_data = pd.read_csv('/Users/Alfredo/Desktop/clustering-stocks/WIKI-datasets-codes.csv', header=-1)
tickers_data.columns = ['Ticker', 'Description']
tickers_data.head(1)

,Ticker,Description
0,WIKI/AAPL,"Apple Inc (AAPL) Prices, Dividends, Splits and..."


In [5]:
tickers_data.shape

(3196, 2)

## Filters and Macthes to available companies in both files

In [6]:
# AddS WIKI/ to Symbol FeatureS
companies_data['Symbol'] = 'WIKI/' + companies_data['Symbol'].astype(str)

# Gives all North America Publicly Traded Companies available through Quandl codes
tickers_data = companies_data.loc[companies_data['Symbol'].isin(tickers_data.Ticker)]
tickers_data.head(1)

,Symbol,Name,LastSale,MarketCap,ADR TSO,IPOyear,Sector,Industry,Summary Quote,Unnamed: 9
0,WIKI/DDD,3D Systems Corporation,9.15,1.041840e+09,NaN,NaN,Technology,Computer Software: Prepackaged Software,http://www.nasdaq.com/symbol/ddd,NaN


In [7]:
tickers_data.shape

(1135, 10)

### Builds Feature(Yrs_Publ_Trded)

In [9]:
tickers = tickers_data.Symbol[:200].tolist()
symbols = []

for sym in tickers:
    
    start = ""
    end = ""   
    mydata = quandl.get(sym, start_date=start, end_date=end, api_key = key)
    mydata['Symbol'] = sym
    symbols.append(mydata)
       
# attach tickers tables
mydata = pd.concat(symbols)

# index 'Date' to column
mydata.reset_index(inplace=True)

# Change location of column 'Symbol'
symb = mydata['Symbol']
mydata.drop(labels=['Symbol'], axis=1,inplace = True)
mydata.insert(1, 'Symbol', symb)

#Merges tickers data features
mydata = mydata.merge(tickers_data)
mydata.head(1)
#mydata.head(1)
#----------------
# Finds the number of years each company has been publicly traded
df_years = mydata.set_index('Symbol')

min_data = df_years.groupby('Symbol')['Date'].min() 
max_data = df_years.groupby('Symbol')['Date'].max()
new_df = (max_data - min_data).to_frame() / 365.25
#new_df.head()
new_df['Date'] = new_df['Date'].map(lambda x: str(x)[:-21])
new_df = new_df.rename(index=str, columns={"Date":"Yrs_Publ_Trded"}).reset_index()
#new_df

mydata_yrs = mydata.merge(new_df)
mydata_yrs = mydata_yrs.groupby(['Symbol','Yrs_Publ_Trded']).size().to_frame().reset_index()
mydata_yrs = mydata_yrs.drop(mydata_yrs.columns[2],axis=1)


data1 = mydata.merge(mydata_yrs)
#data1.head(1)

# Slices WIKI out of symbol column for tickers
data1['Symbol'] = data1['Symbol'].map(lambda x: str(x)[5:])
data1.head(2)

,Date,Symbol,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume,Name,LastSale,MarketCap,ADR TSO,IPOyear,Sector,Industry,Summary Quote,Unnamed: 9,Yrs_Publ_Trded
0,1990-11-05,DDD,3.88,4.25,3.25,4.25,34800.0,0.0,1.0,3.88,4.25,3.25,4.25,34800.0,3D Systems Corporation,9.15,1.041840e+09,NaN,NaN,Technology,Computer Software: Prepackaged Software,http://www.nasdaq.com/symbol/ddd,NaN,27
1,1990-11-06,DDD,3.50,4.25,3.50,3.62,21600.0,0.0,1.0,3.50,4.25,3.50,3.62,21600.0,3D Systems Corporation,9.15,1.041840e+09,NaN,NaN,Technology,Computer Software: Prepackaged Software,http://www.nasdaq.com/symbol/ddd,NaN,27


## Filters historical prices for year given(2016)

In [10]:
#LOOPS THROUGH TICKERS FOR 2016 DAILY RECORDS
symbols = []

for sym in tickers:
    
    start = "2016-01-01"
    end = "2016-12-12"
    
    mydata_2016 = quandl.get(sym, start_date=start, end_date=end, api_key = key)
    mydata_2016['Symbol'] = sym
    symbols.append(mydata_2016)
    
    
# attach tickers tables
mydata_2016 = pd.concat(symbols)

# index 'Date' to column
mydata_2016.reset_index(inplace=True)

# Change location of column 'Symbol'
symb = mydata_2016['Symbol']
mydata_2016.drop(labels=['Symbol'], axis=1,inplace = True)
mydata_2016.insert(1, 'Symbol', symb)
#mydata_2016.head(1)

mydata_2016['Symbol'] = mydata_2016['Symbol'].map(lambda x: str(x)[5:])
#mydata_2016.head(3)

# Merge year features/keeps 2016 prices
data2 = mydata_2016.merge(data1)
data2.head(3)

,Date,Symbol,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume,Name,LastSale,MarketCap,ADR TSO,IPOyear,Sector,Industry,Summary Quote,Unnamed: 9,Yrs_Publ_Trded
0,2016-01-04,DDD,8.47,10.15,8.46,9.83,7612842.0,0.0,1.0,8.47,10.15,8.46,9.83,7612842.0,3D Systems Corporation,9.15,1.041840e+09,NaN,NaN,Technology,Computer Software: Prepackaged Software,http://www.nasdaq.com/symbol/ddd,NaN,27
1,2016-01-05,DDD,9.83,10.24,9.29,9.69,4911636.0,0.0,1.0,9.83,10.24,9.29,9.69,4911636.0,3D Systems Corporation,9.15,1.041840e+09,NaN,NaN,Technology,Computer Software: Prepackaged Software,http://www.nasdaq.com/symbol/ddd,NaN,27
2,2016-01-06,DDD,9.48,9.55,8.99,9.10,3290163.0,0.0,1.0,9.48,9.55,8.99,9.10,3290163.0,3D Systems Corporation,9.15,1.041840e+09,NaN,NaN,Technology,Computer Software: Prepackaged Software,http://www.nasdaq.com/symbol/ddd,NaN,27


In [11]:
data = data2.drop(['Name','LastSale','ADR TSO','IPOyear','Summary Quote','Unnamed: 9'], axis=1)
#data.head(1)

# Changes MarketCap feature from scientific notation to integer value
data['MarketCap'] = data['MarketCap'].apply(lambda x: '%.f' % x)
#data.head()

# resetting indexes
mydata = data.set_index(['Date','Symbol']).unstack(0).stack(dropna=False).groupby(level='Symbol').ffill().bfill()
mydata.head(1)

,,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume,MarketCap,Sector,Industry,Yrs_Publ_Trded
Symbol,Date,,,,,,,,,,,,,,,,
A,2016-01-04,41.06,41.19,40.34,40.69,3287335.0,0.0,1.0,40.346337,40.474077,39.638851,39.982768,3287335.0,21722962316,Capital Goods,Biotechnology: Laboratory Analytical Instruments,18


In [ ]:
# Below section needs updates --------------------------------------------------------------------------------------------------

#### Keeps building features

In [9]:
# Builds daily return column
mydata['Daily_Return'] = mydata['Adj. Close'].pct_change(1)
#mydata.head(1)

# Builds overall percentage on return or Cumulitive return(return difference from start to current price)
mydata['Cumulitive_Return'] = mydata.groupby('Symbol')['Adj. Close'].transform(lambda x: x[-1]/x[0] -1)
#mydata.head(2)

# Calculates daily average high, low, volume trader for each stock
mydata['Avg_High'] = mydata.groupby('Symbol').High.transform('mean')
mydata['Avg_Low'] = mydata.groupby('Symbol').Low.transform('mean')
mydata['Avg_Vol_Trader'] = mydata.groupby('Symbol').Volume.transform('mean')
mydata['Avg_Dividend'] = mydata.groupby('Symbol')['Ex-Dividend'].transform('mean')
#mydata.head(2)

# Mean Daily Return
d_mean_return = mydata.groupby('Symbol').Daily_Return.transform('mean').to_frame()
mydata['Avg_Daily_Return'] = d_mean_return
#mydata.head(2)

# Std Daily
d_std_return = mydata.groupby('Symbol').Daily_Return.transform('std').to_frame()
mydata['Std_Daily_Return'] = d_std_return
#mydata.head(2)

# Calculates Daily Sharpe Ratio(Calculates risk adjusted return)
# mean return - risk free rate / standard deviation
# For this formula we are assuming a risk free rate of 0(different people,different opinions)
sr = d_mean_return/d_std_return
mydata['Daily_Sharpe_Ratio'] = sr

mydata.head(1)

,,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume,Yrs_Publ_Trded,Name,LastSale,MarketCap,ADR TSO,IPOyear,Sector,Industry,Summary Quote,Unnamed: 9,Daily_Return,Cumulitive_Return,Avg_High,Avg_Low,Avg_Vol_Trader,Avg_Dividend,Avg_Daily_Return,Std_Daily_Return,Daily_Sharpe_Ratio
Symbol,Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AA,2016-11-01,22.1,23.55,21.78,23.0,32216510.0,0.0,1.0,22.1,23.55,21.78,23.0,32216510.0,1,Alcoa Corporation,46.52,8.607267e+09,NaN,2016.0,Basic Industries,Aluminum,http://www.nasdaq.com/symbol/aa,NaN,NaN,0.358261,29.779172,28.289038,6.427900e+06,0.0,0.011461,0.031403,0.364976


In [10]:
# Builds list to easy access to reorder
cols = list(mydata.columns.values)
mydata = mydata[['Open','High','Low','Close','Volume','Ex-Dividend','Split Ratio','Adj. Open','Adj. High','Adj. Low',
                 'Adj. Close','Adj. Volume','Daily_Return','Cumulitive_Return','Yrs_Publ_Trded','Avg_High','Avg_Low',
                 'Avg_Vol_Trader','Avg_Dividend','Avg_Daily_Return','Std_Daily_Return','Daily_Sharpe_Ratio']]
mydata.head(1)

,,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume,Daily_Return,Cumulitive_Return,Yrs_Publ_Trded,Avg_High,Avg_Low,Avg_Vol_Trader,Avg_Dividend,Avg_Daily_Return,Std_Daily_Return,Daily_Sharpe_Ratio
Symbol,Date,,,,,,,,,,,,,,,,,,,,,,
AA,2016-11-01,22.1,23.55,21.78,23.0,32216510.0,0.0,1.0,22.1,23.55,21.78,23.0,32216510.0,NaN,0.358261,1,29.779172,28.289038,6.427900e+06,0.0,0.011461,0.031403,0.364976


In [11]:
mydata1 = mydata.drop(mydata.columns[:14], axis=1)
mydata2 = mydata1.reset_index(level=1, drop=True)
mydata3 = mydata2.groupby('Symbol').first()
mydata3

,Yrs_Publ_Trded,Avg_High,Avg_Low,Avg_Vol_Trader,Avg_Dividend,Avg_Daily_Return,Std_Daily_Return,Daily_Sharpe_Ratio
Symbol,,,,,,,,
AA,1,29.779172,28.289038,6.427900e+06,0.0,0.011461,0.031403,0.364976
